In [1]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [2]:
v = '10-2'

In [3]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_mlm3-pmminilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
# pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample'
# pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-oof-ft'
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-oof'

# context model
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.add_probs'
c_model = 'deberta-v3-small.flag-context4-2-d-oof.add_probs'
# c_model = 'deberta-v3-small.flag-context4-2-d-extpred-ft.0804'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-oof-ft-mminilm'

In [4]:
df_train = pd.read_feather('../working/train.fea')
df_train = df_train[df_train.fold==0]

In [5]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

In [6]:
# d = pd.DataFrame(gezi.batch2list(xc))
# dg = d.groupby('pred_id')
# max_preds = dg['pred'].max()
# min_preds = dg['pred'].min()
# max_cls_preds = dg['cls_pred'].max()
# min_cls_preds = dg['cls_pred'].min()

In [6]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
# xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

ic(calc_metric(xp, 'cls_pred'))
ic(calc_metric(xc, 'reg_pred'))
ic(calc_metric(xc, 'cls_pred'))
ic(calc_metric(xpc, 'cls_pred'))

[08/11/22 15:14:24] 603994771.py:9 in <module>
                    calc_metric(xp, 'cls_pred'): 0.8940525259130645
[08/11/22 15:14:34] 603994771.py:10 in <module>
                    calc_metric(xc, 'reg_pred'): 0.9133999180447212
[08/11/22 15:14:42] 603994771.py:11 in <module>
                    calc_metric(xc, 'cls_pred'): 0.9139478196978118
[08/11/22 15:14:50] 603994771.py:12 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.902516912439078


0.902516912439078

In [7]:
pc_preds = dict(zip(xpc['cid'], xpc['pred']))
pc_cls_preds = dict(zip(xpc['cid'], xpc['cls_pred']))
pc_probs = dict(zip(xpc['cid'], xpc['probs']))

preds = []
cls_preds = []
probs = []
marks = []
for cid, pred, cls_pred, prob in zip(xp['cid'], xp['pred'], xp['cls_pred'], xp['probs']):
  # if prob.max() < 10:
  if prob.max() < 0.8:
    marks.append(1)
    preds.append(pc_preds.get(cid, pred))
    probs.append(pc_probs.get(cid, prob))
    cls_preds.append(pc_cls_preds.get(cid, cls_pred))
  else:
    marks.append(0)
    preds.append(pred)
    probs.append(prob)
    cls_preds.append(cls_pred)
    
xpc = xp.copy()
xpc['pred'] = np.asarray(preds)
xpc['cls_pred'] = np.asarray(cls_preds)
xpc['probs'] = np.asarray(probs)
xpc['mark'] = np.asarray(marks)
ic(xpc['mark'].mean())
ic(calc_metric(xpc, 'cls_pred'))

[08/11/22 15:14:54] 2337548337.py:27 in <module>
                    xpc['mark'].mean(): 0.5019261406823974
[08/11/22 15:15:01] 2337548337.py:28 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9026518550887598


0.9026518550887598

In [8]:
LABEL_COL = 'rel_rank'
def diff_feats(x, l,r, name):
  if isinstance(l, str):
    x[f'{name}_diff'] = x[l] - x[r]
  else:
    x[f'{name}_diff'] = l - r
  x[f'abs_{name}_diff'] = abs(x[f'{name}_diff'])

def gen_pfeat(x_, topn=None):
  x = {}
  keys = ['pred', 'cls_pred']
  x = {k: x_[k] for k in keys}
  # x['rank_pred'] = x['pred'] * (1 + x_['n_code_cell']) - 0.5
  # x['cls_rank_pred'] = x['cls_pred'] * (1 + x_['n_code_cell']) - 0.5
  x['min_prob'] = x_['probs'].min()
  x['var_prob'] = np.var(x_['probs'])
  if 'sims' in x_:
    x['min_sim'] = x_['sims'].min()
    x['var_sim'] = np.var(x_['sims'])
  
  idxes = (-x_['probs']).argsort()
  # topn = topn or len(idxes)
  topn = topn or 10
  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / (x_['n_code_cell'] + 1)
        # x[f'top_rank_{i}'] = idxes[i]
      x[f'top_prob_{i}'] = x_['probs'][idxes[i]]
      if 'sims' in x_:
        x[f'top_sim_{i}'] = x_['sims'][idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
      if 'sims' in x_:
        x[f'top_sim_{i}'] = -1
  
  diff_feats(x, 'cls_pred', 'pred', 'cls')
  # diff_feats(x, 'cls_rank_pred', 'rank_pred', 'cls_rank')
  return x

def gen_cfeat(x_):
  topn = min(10, len(x_['cls_pred_ori']))
  x = {}
  keys = [
    # 'pred', 
    'cls_pred', 
    'reg_pred', 
    'cls2_pred', 
    'group_pos',
    'group_rank',
    'group_mean',
    'group_max',
    'group_min',
    'group_var',
    ]
  x = {k: x_[k] for k in keys}
  # x['cls_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  # x['cls2_rank_pred'] = x['cls2_pred'] * FLAGS.num_classes - 0.5
  # x['reg_rank_pred'] = x['reg_pred'] * FLAGS.num_classes - 0.5
  preds = x_['cls_pred_ori']
  probs = gezi.softmax(preds)
  x['min_prob'] = probs.min()
  x['var_prob'] = np.var(probs)
  idxes = (-probs).argsort()

  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / FLAGS.num_classes
        # x[f'tpop_rank_{i}']  = idxes[i] 
      x[f'top_prob_{i}'] = probs[idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
  diff_feats(x, 'cls_pred', 'reg_pred', 'cls_reg')
  diff_feats(x, 'cls_pred', 'cls2_pred', 'cls_cls2')
  diff_feats(x, 'cls2_pred', 'reg_pred', 'cls2_reg')
  # diff_feats(x, 'cls_rank_pred', 'reg_rank_pred', 'cls_reg_rank')
  # diff_feats(x, 'cls_rank_pred', 'cls2_rank_pred', 'cls_cls2_rank')
  # diff_feats(x, 'cls2_rank_pred', 'reg_rank_pred', 'cls2_reg_rank')
  # group_id = x_['pred_id']
  # x['max_pred'] = max_preds[group_id]
  # x['min_pred'] = min_preds[group_id]
  # x['pred_ratio'] = (x['pred'] - x['min_pred']) / (x['max_pred'] - x['min_pred']) if (x['max_pred'] - x['min_pred']) else 1
  # x['max_cls_pred'] = max_cls_preds[group_id]
  # x['min_cls_pred'] = min_cls_preds[group_id]
  # x['cls_pred_ratio'] = (x['cls_pred'] - x['min_cls_pred']) / (x['max_cls_pred'] - x['min_cls_pred']) if (x['max_cls_pred'] - x['min_cls_pred']) else 1
  
  return x

def gen_feats():
  xs = gezi.batch2list(xp)
  p_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp')]
  xs = gezi.batch2list(xc)
  c_feats = [gen_cfeat(x) for x in tqdm(xs, desc=f'gen_cfeats:xc')]
  xs = gezi.batch2list(xpc)
  pc_feats = [gen_pfeat(x, topn=3) for x in tqdm(xs, desc=f'gen_pfeats:xpc')]

  feats = []
  for i in tqdm(range(len(p_feats)), desc='merge_feats'):
    fe1 = p_feats[i]
    fec = c_feats[i]
    fepc = pc_feats[i]
    fepc['mark'] = xpc['mark'][i]
    
    fe = {}
    fe['code_ratio'] = xp['n_code_cell'][i] / xp['n_cell'][i]
    diff_feats(fe, fe1['pred'], fec['reg_pred'], 'pc_reg')
    diff_feats(fe, fe1['pred'], fec['cls_pred'], 'pc_cls')
    diff_feats(fe, fe1['pred'], fec['cls2_pred'], 'pc_cls2')
    diff_feats(fe, fe1['cls_pred'], fec['reg_pred'], 'pc_cls_reg')
    diff_feats(fe, fe1['cls_pred'], fec['cls_pred'], 'pc_cls_cls')
    diff_feats(fe, fe1['cls_pred'], fec['cls2_pred'], 'pc_cls_cls2')
    
    diff_feats(fe, fepc['pred'], fec['reg_pred'], 'pcc_reg')
    diff_feats(fe, fepc['pred'], fec['cls_pred'], 'pcc_cls')
    diff_feats(fe, fepc['pred'], fec['cls2_pred'], 'pcc_cls2')
    diff_feats(fe, fepc['cls_pred'], fec['reg_pred'], 'pcc_cls_reg')
    diff_feats(fe, fepc['cls_pred'], fec['cls_pred'], 'pcc_cls_cls')
    diff_feats(fe, fepc['cls_pred'], fec['cls2_pred'], 'pcc_cls_cls2')
    
    # diff_feats(fe, fe1['rank_pred'], fec['reg_rank_pred'], 'pc_cls_rank')
    # diff_feats(fe, fe1['cls_rank_pred'], fec['reg_rank_pred'], 'pc_cls_reg_rank')
    # diff_feats(fe, fepc['rank_pred'], fec['reg_rank_pred'], 'pcc_cls_rank')
    # diff_feats(fe, fepc['cls_rank_pred'], fec['reg_rank_pred'], 'pcc_cls_reg_rank')

    fe1 = gezi.dict_prefix(fe1, 'xp_')
    fe.update(fe1)
    fec = gezi.dict_prefix(fec, 'xc_')
    fe.update(fec)
    fepc = gezi.dict_prefix(fepc, 'xpc_')
    fe.update(fepc)

    keys = [
     'id', 'cell_id', 'cid',
     'n_words', 'n_code_cell', 'n_cell'
    ]
    for key in keys:
      fe[key] = xp[key][i]
      
    #  ic(feat)
    feats.append(fe)
  #  break
  dfeats = pd.DataFrame(feats)
  return dfeats

In [9]:
dfeats = gen_feats()

gen_pfeats:xp:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_cfeats:xc:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_pfeats:xpc:   0%|          | 0/424943 [00:00<?, ?it/s]

merge_feats:   0%|          | 0/424943 [00:00<?, ?it/s]

In [10]:
df = dfeats.merge(df_train[['cid', 'ancestor_id', LABEL_COL]], on='cid')

In [11]:
gezi.set_fold(df, 5, 'ancestor_id')

In [12]:
reg_cols = [x for x in dfeats.columns if x not in ['id', 'cell_id', 'cid', LABEL_COL]]
cat_cols = []
cols = reg_cols + cat_cols
len(cols)

118

In [13]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [14]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [30]:
from catboost import CatBoostRegressor, CatBoostRanker, CatBoostClassifier
cbt_params = {
              # 'bootstrap_type': 'Poisson',
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              # 'reg_lambda': 7.960622217848342e-07, 
              'l2_leaf_reg': 3,
              'subsample': 0.7422597612762745,
              'bagging_temperature': 0.2,
              # 'max_depth': 10, 
              # 'max_depth': 8,
              'max_depth': 6,
              # 'max_depth': 5,
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              # 'min_child_samples': 100,
              'min_child_samples': 200,
              # 'task_type': 'GPU',
              # 'devices': '0',
              }

In [31]:
cbt_model = CatBoostRegressor(**cbt_params)
cbt_model.fit(X_train, y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=100,
              )  
dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
ic(cbt_score)

0:	learn: 0.2522763	test: 0.2522763	test1: 0.2522183	best: 0.2522183 (0)	total: 47.3ms	remaining: 7m 52s
100:	learn: 0.0658034	test: 0.0658034	test1: 0.0670061	best: 0.0670061 (100)	total: 3.46s	remaining: 5m 39s
200:	learn: 0.0591174	test: 0.0591174	test1: 0.0605267	best: 0.0605267 (200)	total: 6.66s	remaining: 5m 24s
300:	learn: 0.0583037	test: 0.0583037	test1: 0.0597833	best: 0.0597833 (300)	total: 9.87s	remaining: 5m 18s
400:	learn: 0.0576059	test: 0.0576059	test1: 0.0591464	best: 0.0591464 (400)	total: 13.3s	remaining: 5m 17s
500:	learn: 0.0571050	test: 0.0571050	test1: 0.0586870	best: 0.0586870 (500)	total: 16.6s	remaining: 5m 15s
600:	learn: 0.0567123	test: 0.0567123	test1: 0.0583434	best: 0.0583434 (600)	total: 20s	remaining: 5m 13s
700:	learn: 0.0564397	test: 0.0564397	test1: 0.0581017	best: 0.0581017 (700)	total: 23.5s	remaining: 5m 11s
800:	learn: 0.0562348	test: 0.0562348	test1: 0.0579313	best: 0.0579313 (800)	total: 26.8s	remaining: 5m 7s
900:	learn: 0.0560870	test: 0.0560

[08/11/22 15:55:49] 3198672829.py:8 in <module>
                    cbt_score: 0.9199360903192325


0.9199360903192325

In [32]:
gezi.plot.feature_importance(cbt_model, topn=20)

In [33]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          # 'num_leaves': 32,
          # 'num_leaves': 24,
          'num_leaves': 12,
          # 'min_data_in_leaf': 300,
          'min_data_in_leaf': 500,
          # 'max_depth': 5,
          'learning_rate': 0.03,
          # "feature_fraction": 0.8,
          # "bagging_fraction": 0.75,
          # "feature_fraction": 0.9,
          # "bagging_fraction": 0.8,
          'feature_fraction': 0.7,
          'bagging_fraction': 0.7,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

In [34]:
d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

lgb_model = lgb.train(params,
                d_train,
                20000,
                valid_sets=[d_train, d_valid],
                verbose_eval=100,
                early_stopping_rounds=500)
dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
ic(lgb_score)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.067164 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 27681
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 118
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 500 rounds
[100]	training's l1: 0.0667781	valid_1's l1: 0.0679191
[200]	training's l1: 0.0595653	valid_1's l1: 0.060877
[300]	training's l1: 0.0586352	valid_1's l1: 0.0599971
[400]	training's l1: 0.0578357	valid_1's l1: 0.0592616
[500]	training's l1: 0.0572253	valid_1's l1: 0.0587192
[600]	training's l1: 0.0568355	valid_1's l1: 0.0583808
[700]	training's l1: 0.0565811	valid_1's l1: 0.0581562
[800]	training's l1: 0.0563921	valid_1's l1: 0.0579969
[900]	training's l1: 0.05626	valid_1's l1: 0.0578893
[1000]	training's l1: 0.0561188	valid_1's l1: 0.0577714
[1100]	training's l1: 0.0560202	valid_1's 

[08/11/22 16:00:44] 176747773.py:12 in <module>
                    lgb_score: 0.9199717439647083


0.9199717439647083

In [35]:
gezi.plot.feature_importance(lgb_model, topn=20)

In [36]:
FOLDS = 5
cbt_scores, lgb_scores = [], []
scores = []
gezi.try_mkdir(f'../working/trees{v}')
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[LABEL_COL]
  X_valid = dvalid[cols]
  y_valid = dvalid[LABEL_COL]
  if fold > 0:
    cbt_model = CatBoostRegressor(**cbt_params)
    cbt_model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=500,
                  )  
  cbt_model.save_model(f'../working/trees{v}/{fold}.cbt')
  dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  if fold > 0:
    lgb_model = lgb.train(params,
                    d_train,
                    20000,
                    valid_sets=[d_train, d_valid],
                    verbose_eval=500,
                    early_stopping_rounds=200)
  lgb_model.save_model(f'../working/trees{v}/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cbt_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})
  ic(score)
  
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )

  0%|          | 0/5 [00:00<?, ?it/s]

[08/11/22 16:00:56] 1820686574.py:21 in <module>
                    cbt_score: 0.9199360903192325
[08/11/22 16:01:36] 1820686574.py:36 in <module>
                    lgb_score: 0.9199717439647083
[08/11/22 16:01:37] 1820686574.py:38 in <module>
                    score: 0.920239250556202
[08/11/22 16:01:37] 1820686574.py:43 in <module>
                    fold: 0
                    np.asarray(cbt_scores).mean(): 0.9199360903192325
                    np.asarray(lgb_scores).mean(): 0.9199717439647083
                    np.asarray(scores).mean(): 0.920239250556202


0:	learn: 0.2522760	test: 0.2522760	test1: 0.2521873	best: 0.2521873 (0)	total: 50.7ms	remaining: 8m 27s
500:	learn: 0.0574364	test: 0.0574364	test1: 0.0566867	best: 0.0566867 (500)	total: 18.4s	remaining: 5m 49s
1000:	learn: 0.0563435	test: 0.0563435	test1: 0.0558224	best: 0.0558224 (1000)	total: 36.6s	remaining: 5m 28s
1500:	learn: 0.0558728	test: 0.0558728	test1: 0.0555510	best: 0.0555510 (1500)	total: 54.8s	remaining: 5m 10s
2000:	learn: 0.0555130	test: 0.0555130	test1: 0.0553999	best: 0.0553984 (1997)	total: 1m 13s	remaining: 4m 52s
2500:	learn: 0.0552135	test: 0.0552135	test1: 0.0553028	best: 0.0553028 (2500)	total: 1m 31s	remaining: 4m 33s
3000:	learn: 0.0549702	test: 0.0549702	test1: 0.0552414	best: 0.0552414 (3000)	total: 1m 49s	remaining: 4m 14s
3500:	learn: 0.0547180	test: 0.0547180	test1: 0.0552023	best: 0.0552004 (3441)	total: 2m 6s	remaining: 3m 55s
4000:	learn: 0.0545007	test: 0.0545007	test1: 0.0551645	best: 0.0551642 (3995)	total: 2m 24s	remaining: 3m 36s
4500:	learn: 

[08/11/22 16:07:41] 1820686574.py:21 in <module>
                    cbt_score: 0.9224734998224717


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060252 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 27680
[LightGBM] [Info] Number of data points in the train set: 339696, number of used features: 118
[LightGBM] [Info] Start training from score 0.455882
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0576332	valid_1's l1: 0.0569578
[1000]	training's l1: 0.0565116	valid_1's l1: 0.0559804
[1500]	training's l1: 0.0561836	valid_1's l1: 0.0557432
[2000]	training's l1: 0.0559817	valid_1's l1: 0.0556079
[2500]	training's l1: 0.0558485	valid_1's l1: 0.0555356
[3000]	training's l1: 0.0557087	valid_1's l1: 0.0554674
[3500]	training's l1: 0.0556059	valid_1's l1: 0.0554197
[4000]	training's l1: 0.0554738	valid_1's l1: 0.0553688
[4500]	training's l1: 0.0553664	valid_1's l1: 0.0553273
[5000]	training's l1: 0.0552913	valid_1's l1: 0.0552983
[5500]	training's l1: 0.055219	

...

[08/11/22 16:31:09] 1820686574.py:36 in <module>
                    lgb_score: 0.9247171958190019
[08/11/22 16:31:11] 1820686574.py:38 in <module>
                    score: 0.9249285462838717
[08/11/22 16:31:11] 1820686574.py:43 in <module>
                    fold: 3
                    np.asarray(cbt_scores).mean(): 0.9221405729881422
                    np.asarray(lgb_scores).mean(): 0.9221764691413599
                    np.asarray(scores).mean(): 0.9224120517700669


0:	learn: 0.2522343	test: 0.2522343	test1: 0.2523154	best: 0.2523154 (0)	total: 47.5ms	remaining: 7m 55s
500:	learn: 0.0569993	test: 0.0569993	test1: 0.0583458	best: 0.0583458 (500)	total: 18.6s	remaining: 5m 52s
1000:	learn: 0.0559816	test: 0.0559816	test1: 0.0574762	best: 0.0574762 (1000)	total: 37.9s	remaining: 5m 40s
1500:	learn: 0.0555296	test: 0.0555296	test1: 0.0571694	best: 0.0571694 (1500)	total: 56.9s	remaining: 5m 21s
2000:	learn: 0.0552251	test: 0.0552251	test1: 0.0570218	best: 0.0570218 (2000)	total: 1m 15s	remaining: 5m 1s
2500:	learn: 0.0549718	test: 0.0549718	test1: 0.0569065	best: 0.0569065 (2500)	total: 1m 34s	remaining: 4m 42s
3000:	learn: 0.0547246	test: 0.0547246	test1: 0.0568236	best: 0.0568232 (2996)	total: 1m 52s	remaining: 4m 22s
3500:	learn: 0.0545037	test: 0.0545037	test1: 0.0567767	best: 0.0567767 (3500)	total: 2m 10s	remaining: 4m 3s
4000:	learn: 0.0542986	test: 0.0542986	test1: 0.0567187	best: 0.0567187 (4000)	total: 2m 29s	remaining: 3m 44s
4500:	learn: 0

[08/11/22 16:37:14] 1820686574.py:21 in <module>
                    cbt_score: 0.915529650315392


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.063380 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 27679
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 118
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0573532	valid_1's l1: 0.0586091
[1000]	training's l1: 0.0562628	valid_1's l1: 0.0576436
[1500]	training's l1: 0.0558913	valid_1's l1: 0.0573272
[2000]	training's l1: 0.0556995	valid_1's l1: 0.0571852
[2500]	training's l1: 0.0555458	valid_1's l1: 0.0570842
[3000]	training's l1: 0.0554027	valid_1's l1: 0.056995
[3500]	training's l1: 0.0552996	valid_1's l1: 0.0569394
[4000]	training's l1: 0.055191	valid_1's l1: 0.0568764
[4500]	training's l1: 0.0551095	valid_1's l1: 0.0568334
[5000]	training's l1: 0.0550301	valid_1's l1: 0.0567999
[5500]	training's l1: 0.0549551	v

[08/11/22 16:40:59] 1820686574.py:36 in <module>
                    lgb_score: 0.915410297679708
[08/11/22 16:41:01] 1820686574.py:38 in <module>
                    score: 0.9156976199879968
[08/11/22 16:41:01] 1820686574.py:43 in <module>
                    fold: 4
                    np.asarray(cbt_scores).mean(): 0.9208183884535922
                    np.asarray(lgb_scores).mean(): 0.9208232348490295
                    np.asarray(scores).mean(): 0.9210691654136529


In [37]:
# fold: 0
#                     np.asarray(cbt_scores).mean(): 0.9181025338378637
#                     np.asarray(lgb_scores).mean(): 0.9193848140698876
#                     np.asarray(scores).mean(): 0.9195238841373282

#  fold: 4
#                     np.asarray(cbt_scores).mean(): 0.9189608630607822
#                     np.asarray(lgb_scores).mean(): 0.9196959050732708
#                     np.asarray(scores).mean(): 0.9199400102856614